# Configuração do Ambiente

In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# Pré-Processamento

In [ ]:
def get_tipos(path):
  # Leitura do arquivo
  folha = pd.read_excel(path).fillna('')
  # Na terceira linha há os nomes de cada coluna
  folha.columns = folha.iloc[2].tolist()
  # Removendo  linhas de cabeçalho
  folha = folha.iloc[5:]
  folha.reset_index(drop=True, inplace=True)
  # Adicionando coluna de tipo
  nulos = ['Nulo'] * folha.shape[0]
  folha['Tipo'] = nulos
  # Adiciona o nome da empresa:
  # Os arquivos vem na forma 'Nome empresa competencia.xls'
  # Ex: FIBRA NEGOCIOS 012023.xls
  # Então será feito um split() e em seguida um ' '.join()
  # Com todos os elementos menos o último (competencia)
  nome_emp = ' '.join(path.split()[:-1])
  nome_emp = [nome_emp] * folha.shape[0]
  folha['Empresa'] = nome_emp

  # Cada linha deve ter um valor apenas em provento ou em desconto
  # então é feita a verificação de em qual coluna há um valor
  # Em linhas que não há código nem descrição de eventos
  # as duas colunas em questão estarão em branco
  # É também feita uma verificação se há valores nas duas colunas
  # para detecção de anomalias
  for i in range(folha.shape[0]):

    provento = folha.iloc[i]['Provento']
    desconto = folha.iloc[i]['Desconto']

    if provento and desconto:
      folha.loc[i, 'Tipo'] = 'Anomalia'
    elif provento:
      folha.loc[i, 'Tipo'] = 'Provento'
    elif desconto:
      folha.loc[i, 'Tipo'] = 'Desconto'

  # Filtra apenas os valores não nulos
  folha = folha[folha['Tipo'] != 'Nulo']

  # Guarda apenas as colunas desejadas
  folha = folha[['Empresa', 'Evento', 'Descrição', 'Tipo']]

  return folha

# Leitura dos Arquivos

In [ ]:
# Filtragem dos valores do tipo excel
files = [arq for arq in os.listdir() if '.xls' in arq]
files.sort()

folha_total = pd.DataFrame()
# Concatena folha por folha em uma única variável
for arq in tqdm(files):
  folha_temp = get_tipos(arq)
  folha_total = pd.concat([folha_total, folha_temp])

100%|██████████| 44/44 [00:01<00:00, 33.39it/s]


In [ ]:
folha_total.to_excel('Tipos_Eventos.xlsx', index=False)